In [1]:
from pymc_marketing.clv import utils

import pandas as pd

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [24]:
cdnow_trans = "https://raw.githubusercontent.com/pymc-labs/pymc-marketing/main/datasets/cdnow_transactions.csv"

cdnow_df = pd.read_csv(cdnow_trans,dtype={"date":str})
#cdnow_df = cdnow_df.drop(columns={"_id","cds_bought"})

cdnow_df.head()

,_id,id,date,cds_bought,spent
0,4,1,19970101,2,29.33
1,4,1,19970118,2,29.73
2,4,1,19970802,1,14.96
3,4,1,19971212,2,26.48
4,21,2,19970101,3,63.34


`_find_first_transactions` flags the first purchase each customer has made, which must be excluded for modeling. It is called internally by `rfm_summary`

In [25]:
utils._find_first_transactions(
    transactions=cdnow_df, 
    customer_id_col = "id", 
    datetime_col = "date",
    monetary_value_col = "spent", 
    datetime_format = "%Y%m%d",
).head()

,id,date,spent,first
0,1,1997-01-01,29.33,True
1,1,1997-01-18,29.73,False
2,1,1997-08-02,14.96,False
3,1,1997-12-12,26.48,False
4,2,1997-01-01,63.34,True


`rfm_summary` is the primary data preprocessing step for CLV modeling in the continuous, non-contractual domain

In [29]:
rfm_df = utils.rfm_summary(
    cdnow_df, 
    customer_id_col = "id", 
    datetime_col = "date", 
    monetary_value_col = "spent",
    datetime_format = "%Y%m%d",
    time_unit = "W",
    include_first_transaction=False,
)

rfm_df.head()

,customer_id,frequency,recency,T,monetary_value
0,1,3.0,49.0,78.0,23.723333
1,2,1.0,2.0,78.0,11.770000
2,3,0.0,0.0,78.0,0.000000
3,4,0.0,0.0,78.0,0.000000
4,5,0.0,0.0,78.0,0.000000


In [30]:
print(f"Date of first purchase: {cdnow_df['date'].min()}")
print(f"Date of last purchase: {cdnow_df['date'].max()}")

rfm_df.describe()

Date of first purchase: 19970101
Date of last purchase: 19980630


,customer_id,frequency,recency,T,monetary_value
count,2357.000000,2357.000000,2357.000000,2357.000000,2357.000000
mean,1179.000000,1.698770,18.633857,71.985999,18.209059
std,680.551615,3.300726,25.259830,3.346091,37.850023
min,1.000000,0.000000,0.000000,66.000000,0.000000
25%,590.000000,0.000000,0.000000,69.000000,0.000000
50%,1179.000000,0.000000,0.000000,72.000000,0.000000
75%,1768.000000,2.000000,40.000000,75.000000,27.940000
max,2357.000000,36.000000,76.000000,78.000000,1258.462000


For MAP fits and covariate models, `rfm_train_test_split` can be used to evaluate models on unseen data. It is also useful for identifying the impact of a time-based event like a marketing campaign.

In [31]:
utils.rfm_train_test_split(
    cdnow_df, 
    customer_id_col = "id", 
    datetime_col = "date", 
    train_period_end = "19980101",
    monetary_value_col = "spent",
    datetime_format = "%Y%m%d",
    time_unit = "W")

,customer_id,frequency,recency,T,monetary_value,test_frequency,test_monetary_value,test_T
0,1,3.0,49.0,52.0,23.723333,0.0,0.000,26.0
1,2,1.0,2.0,52.0,11.770000,0.0,0.000,26.0
2,3,0.0,0.0,52.0,0.000000,0.0,0.000,26.0
3,4,0.0,0.0,52.0,0.000000,0.0,0.000,26.0
4,5,0.0,0.0,52.0,0.000000,0.0,0.000,26.0
...,...,...,...,...,...,...,...,...
2352,2353,0.0,0.0,40.0,0.000000,1.0,14.490,26.0
2353,2354,5.0,24.0,40.0,44.928000,2.0,19.775,26.0
2354,2355,0.0,0.0,40.0,0.000000,0.0,0.000,26.0
2355,2356,4.0,26.0,40.0,33.317500,1.0,24.600,26.0


In [32]:
cdnow_df["date"] = pd.to_datetime(cdnow_df["date"],format="%Y%m%d")
test_transactions = cdnow_df.loc[cdnow_df["date"] > '1998-01-01'].copy()

test_transactions.head()

,_id,id,date,cds_bought,spent
19,111,6,1998-01-18,4,84.46
20,111,6,1998-02-15,4,123.96
21,111,6,1998-02-21,2,32.98
22,111,6,1998-02-26,2,23.06
23,111,6,1998-05-10,1,72.99


The additional `_id` and `cds_bought` columns had to be dropped when data was loaded because it caused the columns assignment line to break

In [12]:
test_rfm_data = (
        test_transactions.groupby(['id', 'date'], sort=False)
        .agg(lambda r: 1)
        .groupby(level='id')
        .count()
    )
test_rfm_data.columns = ["test_frequency"]
test_rfm_data.head()

ValueError: Length mismatch: Expected axis has 3 elements, new values have 1 elements

In [117]:
test_rfm_data["test_monetary_value"] = (
    test_transactions.groupby(["id", "date"])[
        "spent"
    ]
    .sum()
    .groupby("id")
    .mean()
)

In [124]:
(test_transactions["date"].max().to_period("D") - test_transactions["date"].min().to_period("D")).n

179